In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# DL and data processing:
import torch
from torch import nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class TextFromPandas(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.n = 0
             
    def __getitem__(self, idx):
        text = pd.Series(self.dataframe.iloc[idx, :-1], dtype="string")
        label = self.dataframe.iloc[idx, -1]
        return label, " ".join(text.tolist())
    
    def __len__(self):
        return len(self.dataframe)
    
    def __iter__(self):
        return self;
    
    def __next__(self):
        if self.n < len(self):
            result = self[self.n]
            self.n += 1
            return result
        else:
            raise StopIteration
            

In [3]:
tweet_df = pd.read_csv('data/train.csv').drop(['id'], axis=1)
tweet_df.fillna(value="<unk>", inplace=True)
dataset = TextFromPandas(tweet_df)
tweet_df.head(5)

,keyword,location,text,target
0,<unk>,<unk>,Our Deeds are the Reason of this #earthquake M...,1
1,<unk>,<unk>,Forest fire near La Ronge Sask. Canada,1
2,<unk>,<unk>,All residents asked to 'shelter in place' are ...,1
3,<unk>,<unk>,"13,000 people receive #wildfires evacuation or...",1
4,<unk>,<unk>,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
dataset[0], dataset[-1]

((1,
  '<unk> <unk> Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'),
 (1,
  '<unk> <unk> The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'))

In [5]:
# Build vocabulary from tweets plus AG_NEWS
import itertools
from torchtext.datasets import AG_NEWS

# split an input string into tokens
tokenizer = get_tokenizer('basic_english')

# generator that tokenizes each element of an iterator into a list
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

#join tweets dataset with AG_NEWS dataset to get larger vocabulary
all_text = itertools.chain(dataset, AG_NEWS(split='train'))

# build vocabulary
vocab = build_vocab_from_iterator(yield_tokens(all_text), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [6]:
# Setup collate function and dataloader
text_pipeline = lambda x: vocab(tokenizer(x))

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


In [7]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [8]:
vocab_size = len(vocab)
num_classes = 1
emb_size = 64

model = TextClassificationModel(vocab_size, emb_size, num_classes)

In [9]:
def train_one_epoch(model, opt, criterion, lr, trainloader):
    model.to(device)
    model.train()
    
    running_tl = []
    
    for (label, text, offsets) in trainloader:
        
        opt.zero_grad()
        
        label, text = label.to(device), text.to(device)
        label.reshape(emb_size, 1)
        
        output = model(text, offsets)
        loss = criterion(output, label)
        
        running_tl += loss.item()
        loss.backward()
        opt.step()
        
    return running_tl
        
def validate_one_epoch(model, opt, criterion, lr, validloader):
    
    running_vl = []
    
    model.eval()
    with torch.no_grad():
        for (label, text, offsets) in validloader:
            output = model(text, offsets)
            loss = criterion(output, label)
            running_vl += loss.item()
            
    return running_vl
    

def train_model(model, opt, criterion, lrainload, testload=None, lr=1e-3)
    

In [9]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        
        label = label.reshape(emb_size, 1)
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        print(label.dtype, predicted_label.dtype)
        print(label.shape, predicted_label.shape)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [10]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

# make test/train split:
np.random.seed(1234)
train_split = int(0.8 * len(tweet_df))
idx = np.arange(0, len(tweet_df))
np.random.shuffle(idx)

train_idx = idx[:train_split]
test_idx  = idx[train_split:]

train_data = TextFromPandas(tweet_df.iloc[train_idx])
test_data = TextFromPandas(tweet_df.iloc[test_idx])

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(test_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

torch.int64 torch.float32
torch.Size([64, 1]) torch.Size([64, 1])


RuntimeError: result type Float can't be cast to the desired output type Long